In [1]:
import requests
import os
import zipfile
import shutil
from dotenv import load_dotenv

def get_passenger_volume(api_url, api_key, folder_name, date=None):
    """
    Fetches a link from the LTA DataMall and downloads the passenger volume CSV file.
    
    Parameters:
    - api_url (str): The API endpoint URL for passenger volume data.
    - api_key (str): Your LTA DataMall API key.
    - date (str): Optional Date parameter in the format 'YYYYMM' (e.g., '202108' for August 2021).
    - folder_name (str): Folder name where the extracted CSV should be saved.
    
    Returns:
    - str: Path to the downloaded CSV file.
    """
    headers = {
        'AccountKey': api_key,
        'accept': 'application/json'
    }

    params = {'Date': date} if date else {}
    
    response = requests.get(api_url, headers=headers, params=params)
    print(f"Response: {response.text}")
    
    if response.status_code == 200:
        data = response.json()
        if 'value' in data:
            download_link = data['value'][0]['Link']  # Adjust index if there are multiple links
            
            # Download the zip file from the link
            zip_response = requests.get(download_link, stream=True)
            if zip_response.status_code == 200:
                # Ensure the folder exists
                os.makedirs(folder_name, exist_ok=True)
                
                # Save the zip file
                zip_file_path = os.path.join(folder_name, f'transport_node_bus_{date}.zip')
                with open(zip_file_path, 'wb') as zip_file:
                    zip_file.write(zip_response.content)
                print(f"Zip file downloaded and saved to: {zip_file_path}")
                
                # Unzip the file
                with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
                    zip_ref.extractall(folder_name)
                
                # Get the CSV file name (assumes only one CSV is extracted)
                csv_file_name = f'transport_node_bus_{date}.csv'
                csv_file_path = os.path.join(folder_name, csv_file_name)
                
                # Delete the zip file if you no longer need it
                os.remove(zip_file_path)
                print(f"CSV file extracted and saved to: {csv_file_path}")
                
                return csv_file_path
            else:
                print(f"Failed to download the zip file: {zip_response.status_code}")
                return None
        else:
            print("No 'value' key found in the response.")
            return None
    else:
        print(f"Failed to retrieve data: {response.status_code}")
        return None

